# Rulează pipeline-ul (fără Python local)

Acest notebook rulează pașii: generate → combine → split → train → evaluate în mediul Google Colab (browser).
Instrucțiuni: fie clonezi un repo public GitHub, fie încarci arhiva cu proiectul (zip) în Colab.

## Option A — Clone from GitHub (dacă repository este public)
Înlocuiește `<GIT_URL>` cu URL-ul repo-ului tău și rulează celula.

In [ ]:
# Example: git clone <GIT_URL> repo && cd repo
!git clone <GIT_URL> repo || echo 'skip clone (replace <GIT_URL> if needed)'
%bash
if [ -d repo ]; then
  cd repo
fi

## Option B — Upload repository archive (zip)
Dacă repo-ul nu este pe GitHub, zipează proiectul local și încarcă arhiva aici (Upload).

In [ ]:
from google.colab import files
print('If you want to upload a zip of the project, run the next line and upload the file')
# uploaded = files.upload()
# then unzip: !unzip -q uploaded_filename.zip -d repo

## Install dependencies (use tensorflow-cpu to avoid version issues)
Notebook will try to make a safe requirements file by replacing `tensorflow` with `tensorflow-cpu==2.13.0`.

In [ ]:
%bash
set -e
# assume current dir contains requirements.txt (or under repo/)
REQ_PATH=requirements.txt
if [ -f repo/requirements.txt ]; then REQ_PATH=repo/requirements.txt; fi
if [ -f $REQ_PATH ]; then
  sed -e 's/^tensorflow$/tensorflow-cpu==2.13.0/' $REQ_PATH > /tmp/colab_req.txt || cp $REQ_PATH /tmp/colab_req.txt
  python -m pip install --upgrade pip setuptools wheel
  pip install -r /tmp/colab_req.txt
else
  echo 'requirements.txt not found; please ensure repo is present or upload the project.'
fi

## Run the pipeline
Aceste comenzi presupun că fișierele `src/...` sunt în directorul curent sau sub `repo/`.

In [ ]:
%bash
set -e
if [ -d repo ]; then cd repo; fi
python src/data_acquisition/generate.py --count 500 || true
python src/preprocessing/combine_datasets.py || true
python src/preprocessing/data_splitter.py || true
python src/neural_network/train.py || true
python src/neural_network/evaluate.py || true

In [ ]:
# Show results
%bash
if [ -d repo ]; then cd repo; fi
ls -R . | sed -n '1,200p'
if [ -f results/test_metrics.json ]; then echo 'Test metrics:' && cat results/test_metrics.json; fi
if [ -f models/trained_model.h5 ]; then echo 'Trained model saved at models/trained_model.h5'; fi